In [ ]:
import pandas as pd
from INEvalidador import Validador
from INEvalidador.conexionSQL import baseSQL
import re
import datetime
import dask.dataframe as dd
import os 

conexion_sql = baseSQL()

In [ ]:
# Función que obtenga la primera variable en la validación para poder identificar qué pasa usará
def identificar_variable(condicion: str):
    variable = re.findall(r'^\s*([A-Za-z_][A-Za-z\d_]*)\s*[<>!=]*', condicion, re.MULTILINE)
    return variable

In [ ]:
# Función para convertirlas todas las columnas de la base a mayuscula
def columnas_a_mayuscula(tabla:str):
    base = pd.read_feather(tabla)
    columnas_originales = base.columns
    columnas_nuevas = []
    for columna in columnas_originales:
        col = columna.upper()
        columnas_nuevas.append(col)
    diccionario = dict(zip(columnas_originales, columnas_nuevas))
    base = base.rename(columns=diccionario)
    return base

In [ ]:
def clasificar_validaciones(condiciones:str, bases:list):
    validaciones = pd.read_excel(condiciones)
    df1 = columnas_a_mayuscula(bases[0])
    df2 = columnas_a_mayuscula(bases[1])
    df3 = columnas_a_mayuscula(bases[2])
    for validacion in list(validaciones["Condición o Criterio"]):
        # Asegurarse de que validacion sea una cadena
        validacion_str = str(validacion)
        var = identificar_variable(validacion_str)
        if var in list(df1.columns):
            validacion["Base_a_usar"] = 1
        if var in list(df2.columns):
            validacion["Base_a_usar"] = 2
        if var in list(df3.columns):
            validacion["Base_a_usar"] = 3
    for i in range(len(bases)):
        validaciones[validaciones["Base_a_usar"] == i].to_excel(f"ExpresionesBase{i}.xlsx")


In [ ]:
def main(condiciones, bases: list=["Bases/Ronda1/personas.feather","Bases/Ronda1/HogaresRonda1.feather","Bases/Ronda2/HogaresRonda2.feather"]):
    conexion_sql.obtener_datos()
    clasificar_validaciones(condiciones,bases)
    marca_temp = datetime.now().strftime("%Y%m%d%H%M%S")
    carpeta_padre = "Inconsistencias_{}".format(marca_temp)
    for i in range(len(bases)):
        validador = Validador(bases[i],f'ExpresionesBase{i}.xlsx')
        validador.process_to_export(f'Parte{i}')
    dfs = []
    for i in range(len(bases)):
        dfs.append(pd.read_excel(f'ExpresionesBase{i}.xlsx'))
    df_exportacion = pd.concat(dfs)
    df_exportacion.to_excel(os.path.join(carpeta_padre, f'Inconsistencias.xlsx'),index=False)

In [ ]:
main("Expresiones.xlsx")

In [ ]:
# Función para convertirlas todas las columnas de la base a mayuscula
def columnas_a_mayuscula(df: pd.DataFrame):
    columnas_originales = df.columns
    columnas_nuevas = []
    for columna in columnas_originales:
        col = columna.upper()
        columnas_nuevas.append(col)
    diccionario = dict(zip(columnas_originales, columnas_nuevas))
    df = df.rename(columns=diccionario)
    return df

In [ ]:
import os
import pandas as pd

# Diccionario para almacenar los nombres de los archivos y las columnas
columnas_por_archivo = {}

# Directorio donde se encuentran los archivos Feather
directorio = 'db'

# Recorre todos los archivos en el directorio especificado
for archivo in os.listdir(directorio):
    if archivo.endswith('.feather'):  # Verifica si el archivo es un archivo Feather
        ruta_completa = os.path.join(directorio, archivo)
        try:
            # Lee el archivo Feather
            df = pd.read_feather(ruta_completa)
            df = columnas_a_mayuscula(df)
            
            # Obtiene las columnas del DataFrame
            columnas = df.columns.tolist()

            
            # Agrega el nombre del archivo y las columnas al diccionario
            columnas_por_archivo[archivo] = columnas
        except Exception as e:
            print(f"Error al leer el archivo {archivo}: {str(e)}")

# Muestra el diccionario con los nombres de los archivos y las columnas
columnas_por_archivo


In [ ]:
df_col = conexion_sql.base_df_col

In [ ]:
variables = ['']